In [ ]:
#import packages
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
#import seaborn as sns

In [ ]:
#import data from csv
dataVR1 = pd.read_excel("\path\to\folder",sheet_name="Raw")
dataSeg1 = pd.read_excel("\path\to\folder")

In [ ]:
#clean imported data
#--- VR Data ---
selectVR = [0,1,2,4,5,6,7]
dataVR = dataVR1.iloc[:,selectVR]
columsVR = ['period','product_long','GRID_EUP','revenue','product_short','product_group','FM_client']
dataVR.columns = columsVR
dataVR = dataVR[dataVR['period']==201712]
#--- Seg Data ---
selectSeg = [0,1,2,3,5,7,8,9]
dataSeg = dataSeg1.iloc[:,selectSeg]
columsSeg = ['segment','clientName','GRID_EUP','pam','pamLoc','sector','region','valueProp']
dataSeg.columns = columsSeg

In [ ]:
print('---VR---')
print(dataVR.head(5))
print('---Seg---')
print(dataSeg.head(5))

In [ ]:
# Create merged dataframe
fullData = pd.merge(dataVR, dataSeg, how="inner", on="GRID_EUP")

In [ ]:
# # Graph 1 - FM Potential mapping
# YesFM = fullData[fullData['product_group'] == 'FM']
# YesFMEUP = YesFM['GRID_EUP'].unique()

# fullData1 = fullData.assign(FMclient=2)

# runEnd = fullData1.shape[0]

# for i in range(runEnd) :
#     if fullData1.iloc[i,2] in YesFMEUP :
#         fullData1.loc[i,'FMclient'] = 1
#     else :
#         fullData1.loc[i,'FMclient'] = 0

In [ ]:
YesFM = fullData[fullData['product_group'] == 'FM']
YesFMEUP = YesFM['GRID_EUP'].unique()

FMSplit = fullData
FMSplit = FMSplit.assign(FMclient=0)
FMSplit.loc[FMSplit['GRID_EUP'].isin(YesFMEUP), 'FMclient'] = 1
FMSplit = FMSplit.groupby(by=['segment','FMclient'], as_index=False).agg({'GRID_EUP': pd.Series.nunique})

FMSplitP = pd.pivot_table(FMSplit,index=['segment'],values=['GRID_EUP'], columns=['FMclient'],aggfunc=[np.sum],fill_value=0)
print(FMSplitP)

In [ ]:
#Export Data graph 1
FMSplitP.to_excel('graph1DataFM.xlsx')

In [ ]:
# Graph 2 - Revenue section Potential mapping
FMGraph2Data = fullData
FMGraph2Data = FMGraph2Data.assign(FMclient=0)
FMGraph2Data.loc[FMGraph2Data['GRID_EUP'].isin(YesFMEUP), 'FMclient'] = 1
FMGraph2Data = FMGraph2Data.groupby(['GRID_EUP','clientName','segment','FM_client','FMclient'],as_index=False)[['revenue']].sum()


FMGraph2Rev = FMGraph2Data.groupby(['GRID_EUP'],as_index=False)[['revenue']].sum()
FMFullData = pd.merge(FMGraph2Data, FMGraph2Rev, how="inner", on="GRID_EUP")
FMFullData['revShareFM'] = FMFullData['revenue_x'] / (FMFullData['revenue_y']+0.001)

FMRevShare = FMFullData[FMFullData['FM_client'] == 'yes']
FMRevShareTop = FMRevShare[FMRevShare['revShareFM'] >= 0.5]
FMRevShareMid = FMRevShare[np.logical_and(FMRevShare['revShareFM'] >= 0.25,FMRevShare['revShareFM'] < 0.50)]
FMRevShareLow = FMRevShare[FMRevShare['revShareFM'] < 0.25]

#Result
allEUP = fullData['GRID_EUP'].unique()
FMEUP = FMFullData['GRID_EUP'].unique()

FMTopEUP = FMRevShareTop['GRID_EUP'].unique()
FMMidEUP = FMRevShareMid['GRID_EUP'].unique()
FMLowEUP = FMRevShareLow['GRID_EUP'].unique()

ShareFM = float(FMEUP.size)/float(allEUP.size)
ShareTop = float(FMTopEUP.size)/float(FMEUP.size)
ShareMid = float(FMMidEUP.size)/float(FMEUP.size)
ShareLow = float(FMLowEUP.size)/float(FMEUP.size)


print(ShareFM)
print'---split---'
print(ShareTop)
print(ShareMid)
print(ShareLow)
print(ShareTop+ShareMid+ShareLow)

index = ['all']
columns = ['Zero_FM_Rev', 'Till25_FM_Rev','25_50_FM_Rev','Up50_FM_Rev']
FMRevDis = pd.DataFrame(index=index, columns=columns)
FMRevDis.loc[:,'Zero_FM_Rev'] = float(FMEUP.shape[0]) - float(FMTopEUP.shape[0]) - float(FMMidEUP.shape[0])-float(FMLowEUP.shape[0])
FMRevDis.loc[:,'Till25_FM_Rev'] = float(FMLowEUP.shape[0])
FMRevDis.loc[:,'25_50_FM_Rev'] = float(FMMidEUP.shape[0])
FMRevDis.loc[:,'Up50_FM_Rev'] = float(FMTopEUP.shape[0])

In [ ]:
# Create FM Share Rev
FMRevDis.to_excel('graph2DataFM.xlsx')

In [ ]:
# FMRevShare = FMRevShare.sort_values(by='revenue_x', ascending=False)
# FMRevShare['runningTotal'] = 1
# FMRevShare.iloc[0,8] = FMRevShare.iloc[0,6]


# end = FMRevShare.shape[0]

# for j in range(1,end) :
#     k = j-1
#     FMRevShare.iloc[j,8] = FMRevShare.iloc[j,6] + FMRevShare.iloc[j-1,8]


In [ ]:
# #end-1
# FMRevShare['totRevShareFM'] = 0.000000
# FMRevShare['totRevShareFM'] = FMRevShare.iloc[:,8]/FMRevShare.iloc[-1,8]
# topRevCust = FMRevShare[FMRevShare['totRevShareFM']<0.8]
# topRevCust = topRevCust.sort_values(by='totRevShareFM', ascending=True)

# YesFMTop = topRevCust['GRID_EUP'].unique()

# fullData3 = fullData.assign(FMTopclient=0)

# runEnd3 = fullData3.shape[0]

# for i in range(runEnd3) :
#     if fullData3.iloc[i,2] in YesFMTop :
#         fullData3.loc[i,'FMTopclient'] = 1
    
# print(topRevCust.head(5))

In [ ]:
# graph2_2DataFM = fullData3.groupby(by=['segment','product_group'], as_index=False).agg({'revenue': pd.Series.sum})
# graph2_3DataFM = fullData3.groupby(by=['segment','product_short'], as_index=False).agg({'revenue': pd.Series.sum})

# graph2_2DataFM.to_excel('graph2_3DataFM.xlsx')
# graph2_3DataFM.to_excel('graph2_3DataFM.xlsx')

In [ ]:
# print(dataVR_sub.head(5))

In [ ]:
dataVR_sub = pd.read_excel("\path\to\folder",sheet_name="FM")
dataSeg_sub = pd.read_excel("\path\to\folder",sheet_name="Party_Search_Report_20180523130")

In [ ]:
FM_Data_subLevel = pd.merge(dataSeg_sub, dataVR_sub, how='inner', left_on=['GRID ID'], right_on=['Client Grid ID_LfL'])

In [ ]:
# Filter 1 k
FM_Data_subLevel = FM_Data_subLevel[FM_Data_subLevel['Total Income YTD']>1]

# Revenue buckets
FM_Rev_Data = FM_Data_subLevel.groupby(by=['Client EC UP Grid ID'], as_index=False).agg({'Total Income YTD': pd.Series.sum})
FM_Rev_Data = FM_Rev_Data.assign(FMRevBucket='unknown')
FM_Rev_Data.loc[FM_Rev_Data['Total Income YTD']<=10.0 , 'FMRevBucket'] = '0-10 Seg'
FM_Rev_Data.loc[(FM_Rev_Data['Total Income YTD']>10.0)& (FM_Rev_Data['Total Income YTD']<=50.0) , 'FMRevBucket'] = '10-50 Seg'
FM_Rev_Data.loc[(FM_Rev_Data['Total Income YTD']>50.0)& (FM_Rev_Data['Total Income YTD']<=100.0) , 'FMRevBucket'] = '50-100 Seg'
FM_Rev_Data.loc[FM_Rev_Data['Total Income YTD']>100.0 , 'FMRevBucket'] = '>100 Seg'
# Count Unique country per EUP
FM_Country_Data = FM_Data_subLevel.groupby(by=['Client EC UP Grid ID'], as_index=False).agg({'Country (Res) Description': pd.Series.nunique})
FM_Country_Data = FM_Country_Data.assign(FMCountryBucket='unknown')
FM_Country_Data.loc[FM_Country_Data['Country (Res) Description']<=1 , 'FMCountryBucket'] = '1 country'
FM_Country_Data.loc[(FM_Country_Data['Country (Res) Description']>1)& (FM_Country_Data['Country (Res) Description']<=3) , 'FMCountryBucket'] = '2-3 countries'
FM_Country_Data.loc[(FM_Country_Data['Country (Res) Description']>3)& (FM_Country_Data['Country (Res) Description']<=6) , 'FMCountryBucket'] = '4-6 countries'
FM_Country_Data.loc[FM_Country_Data['Country (Res) Description']>=7 , 'FMCountryBucket'] = '>7 countries'
#Merge data
FM_Country_Data = FM_Country_Data.assign(dummy=1)
FMGraph33Data = pd.merge(FM_Rev_Data, FM_Country_Data, how='inner', on='Client EC UP Grid ID')
FMGraph3Data = FMGraph33Data.groupby(by=['FMRevBucket', 'FMCountryBucket'], as_index=False).agg({'Client EC UP Grid ID': pd.Series.nunique})
FMGraph3Data = pd.pivot_table(FMGraph3Data,index=['FMCountryBucket'],values=['Client EC UP Grid ID'], columns=['FMRevBucket'],aggfunc=[np.sum],fill_value=0)



In [ ]:
FMGraph3Data.to_excel('graph3DataFM.xlsx')

In [ ]:
testDummy = pd.merge(dataSeg_sub, dataVR_sub, how='inner', left_on=['GRID ID'], right_on=['Client Grid ID_LfL'])
testDummy = testDummy.groupby(by=['Client EC UP Grid ID'], as_index=False).agg({'Client Grid ID_LfL': pd.Series.nunique})
print(testDummy.shape)

In [ ]:
totalCompanies = FMGraph33Data.groupby(by=['FMRevBucket'], as_index=False).agg({'Country (Res) Description': pd.Series.mean})

In [ ]:
print(totalCompanies)
print('-----')
print(FMGraph33Data['Country (Res) Description'].mean())